In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
total_players =purchase_data["SN"].nunique()
# purchase_data["SN"].nunique()
print(f"Player count is {total_players}")
print(total_players)

Player count is 576
576


In [3]:
unique_items = len(purchase_data["Item ID"].unique())
avg_price = purchase_data["Price"].mean()
num_purchases = purchase_data["SN"].count()
tot_rev = purchase_data["Price"].sum()
print(unique_items)
print(avg_price)
print(num_purchases)
print(tot_rev)

179
3.050987179487176
780
2379.77


In [4]:
# Place all calculated data in a summary DF
summary_df = pd.DataFrame({"Number of Unique Items": [unique_items], 
                           "Average Price": avg_price, 
                           "Number of Purchases": num_purchases,"Total Revenue": tot_rev})
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [5]:
# calculate percentage and count of gender
gender = purchase_data.groupby("Gender")["SN"].nunique()
gender = gender.to_frame()

percent_gender = gender["SN"]/total_players
gender["Percentage of Players"] = percent_gender
gender = gender.rename(columns={"SN":"Total Count"})

female_total = gender.iloc[0,0]
male_total = gender.iloc[1,0]
other_total = gender.iloc[2,0]

# print(gender.iloc[1,1])
# print(female)
# print(male)
gender


,Total Count,Percentage of Players
Gender,,
Female,81,0.140625
Male,484,0.840278
Other / Non-Disclosed,11,0.019097


In [6]:
print(female_total, total_players)

81 576


In [7]:
purchase_count_df = purchase_data.groupby("Gender")["SN"].count()
avg_price_df = purchase_data.groupby("Gender")["Price"].mean()
total_value_df = purchase_data.groupby("Gender")["Price"].sum()

# purchase_count_df
# avg_price_df
merge1_df = pd.merge(purchase_count_df,avg_price_df, on="Gender")
merge2_df = pd.merge(merge1_df,total_value_df, on="Gender")
merge3_df = pd.merge(merge2_df, gender, on="Gender")
merge3_df

,SN,Price_x,Price_y,Total Count,Percentage of Players
Gender,,,,,
Female,113,3.203009,361.94,81,0.140625
Male,652,3.017853,1967.64,484,0.840278
Other / Non-Disclosed,15,3.346000,50.19,11,0.019097


In [8]:
# Calculate avg purchase per person
avg_purchase = (merge3_df["Price_y"] / merge3_df["Total Count"])
merge3_df["Avg_pur_per_person"] = avg_purchase
merge3_df

,SN,Price_x,Price_y,Total Count,Percentage of Players,Avg_pur_per_person
Gender,,,,,,
Female,113,3.203009,361.94,81,0.140625,4.468395
Male,652,3.017853,1967.64,484,0.840278,4.065372
Other / Non-Disclosed,15,3.346000,50.19,11,0.019097,4.562727


In [9]:
final_merge = merge3_df.rename(columns={"SN_x": "Purchase Count","Price_x": "Average Purchase Price","Price_y": "Total Purchase Price","SN_y": "Total Gender Count","Avg_pur_per_person": "Avg Total Purchase per Person"})
final_merge = final_merge.drop(final_merge.columns[[3,4]], axis=1)
format_dict = {"Average Purchase Price": "${0:,.2f}", "Total Purchase Price": "${0:,.2f}","Avg Total Purchase per Person": "${0:,.2f}","Percentage of Players": "{:.2%}"}
final_merge.style.format(format_dict)

,SN,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [10]:
# Create the bins in which Data will be held
# Bins are 0, 10, 15, 20, 25, 30, 35, 40, 100   
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]

# Create the names for the five bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [11]:
purchase_data["Age Bin"] = pd.cut(purchase_data["Age"], bins, labels=group_names, include_lowest=True)
purchase_data
# df["Test Score Summary"] = pd.cut(df["Test Score"], bins, labels=group_names, include_lowest=True)
# df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,92,Final Critic,4.19,<10


In [12]:
age_df = purchase_data.groupby("Age Bin")
bin_df = age_df.nunique()
perc_player = (bin_df["SN"] / 576)
bin_df["Percentage of Players"] = perc_player
format_dict = {"Percentage of Players": "{:.2%}"}
bin_df.style.format(format_dict)
bin_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin,Percentage of Players
Age Bin,,,,,,,,,
<10,23,17,3,2,21,21,21,1,0.029514
10-14,28,22,5,2,24,24,24,1,0.038194
15-19,136,107,5,3,96,96,83,1,0.185764
20-24,365,258,5,3,163,163,135,1,0.447917
25-29,101,77,5,3,78,78,73,1,0.133681
30-34,73,52,5,2,60,60,56,1,0.090278
35-39,41,31,5,3,37,37,34,1,0.053819
40+,13,12,6,2,13,13,13,1,0.020833


In [13]:
final_bin = bin_df.drop(bin_df.columns[[0,2,3,4,5,6,7]], axis=1)

final_bin = final_bin.rename(columns={"SN": "Total Count"})
format_dict = {'Percentage of Players': '{:.2%}'}
final_bin.style.format(format_dict)
final_bin["Percentage of Players"] = final_bin["Percentage of Players"].map("{: .2%}".format)
final_bin

,Total Count,Percentage of Players
Age Bin,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [14]:
purchase_count_df = purchase_data.groupby("Age Bin")["Price"].count()
avg_price_df = purchase_data.groupby("Age Bin")["Price"].mean()
total_value_df = purchase_data.groupby("Age Bin")["Price"].sum()
# purchase_count_df
# avg_price_df
# total_value_df
merge4_df = pd.merge(purchase_count_df,avg_price_df, on="Age Bin")
merge5_df = pd.merge(merge4_df,total_value_df, on="Age Bin")
avg_total_pp = merge5_df["Price"] / bin_df["SN"]
merge5_df["Avg_pur_per_person"] = avg_total_pp
merge5_df

,Price_x,Price_y,Price,Avg_pur_per_person
Age Bin,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [15]:
purchase_analysis_df = merge5_df.rename(columns={"Price_x": "Purchase Count","Price_y": "Average Purchase Price","Price": "Total Purchase Price","Avg_pur_per_person": "Avg Total Purchase per Person"})
purchase_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
Age Bin,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [16]:
purchase_analysis_df["Average Purchase Price"] = purchase_analysis_df["Average Purchase Price"].map("${0:,.2f}".format)
purchase_analysis_df["Total Purchase Price"] = purchase_analysis_df["Total Purchase Price"].map("${0:,.2f}".format)
purchase_analysis_df["Avg Total Purchase per Person"] = purchase_analysis_df["Avg Total Purchase per Person"].map("${0:,.2f}".format)


purchase_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
Age Bin,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [17]:
purchasect = purchase_data.groupby("SN")["Purchase ID"].nunique()
avgpurchase = purchase_data.groupby("SN")["Price"].mean()
totalvalue = purchase_data.groupby("SN")["Price"].sum()

top_spenders_df = pd.DataFrame({"Purchase Count": purchasect, "Average Purchase Price": avgpurchase, "Total Purchase Value": totalvalue})


top_spenders_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Adairialis76,1,2.280000,2.28
Adastirin33,1,4.480000,4.48
Aeda94,1,4.910000,4.91
Aela59,1,4.320000,4.32
Aelaria33,1,1.790000,1.79
...,...,...,...
Yathecal82,3,2.073333,6.22
Yathedeu43,2,3.010000,6.02
Yoishirrala98,1,4.580000,4.58


In [22]:
# Sort based on total value
sorted_df = top_spenders_df.sort_values("Total Purchase Value", ascending=False)

sorted_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


In [19]:
# format top spenders df
# sorted_df["Average Purchase Price"] = sorted_df["Average Purchase Price"].map("${0:,.2f}".format)
# sorted_df["Total Purchase Price"] = sorted_df["Total Purchase Price"].map("${0:,.2f}".format)

# sorted_df

In [20]:
# extract 3 columns from original DF and create new DF
popular_df = purchase_data[["Item ID","Item Name","Price"]]
popular_df

,Item ID,Item Name,Price
0,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,143,Frenzied Scimitar,1.56
2,92,Final Critic,4.88
3,100,Blindscythe,3.27
4,131,Fury,1.44
...,...,...,...
775,60,Wolf,3.54
776,164,Exiled Doomblade,1.63
777,67,"Celeste, Incarnation of the Corrupted",3.46
778,92,Final Critic,4.19


In [21]:
  = converted_ufo.groupby(['Ite', 'state'])

SyntaxError: invalid syntax (<ipython-input-21-447d9b16c4d3>, line 1)